In [13]:
import openai 
import json 
import os
from dotenv import load_dotenv
from functions_v2 import *

In [2]:
# Load environment variables from .env file
load_dotenv()

# Access environment variables
openai.api_key = os.getenv('OPENAI_API_KEY')

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

In [16]:
query = f"""
summarize the context"""

create_connection()
read_pdf('/Users/galampley/Desktop/ChatYourDocs/TheNeedToRead_Graham.pdf')
learn_document('/Users/galampley/Desktop/ChatYourDocs/TheNeedToRead_Graham.pdf', 'TheNeedToRead_Graham', 'pdf')
original_answer = Answer_from_documents(query)
original_answer

'The context is a passage discussing the importance of reading and writing. It highlights that reading not only imparts knowledge but also teaches writing. The passage emphasizes that writing is a way to convey ideas and also a means of discovering new things. It suggests that writing is essential for solving complex problems and developing ideas. The author asserts that good reading and writing skills are interconnected and vital for effective thinking.'

In [17]:
necessary_info = {
    'query': query,
    'context': read_pdf('/Users/galampley/Desktop/ChatYourDocs/TheNeedToRead_Graham.pdf')
}

In [20]:
def eval_with_rubric(test_set, original_answer):

    original_query = test_set['query']
    context = test_set['context']
    completion = original_answer
    
    system_message = """\
    You are an assistant that evaluates how well the agent \
    answers a user question by looking at the context that the \
    agent is using to generate its response. 
    """

    user_message = f"""\
    You are evaluating a submitted answer to a question based on the context \
    that the agent uses to answer the question.
    Here is the data:
        [BEGIN DATA]
        ************
        [Question]: {original_query}
        ************
        [Context]: {context}
        ************
        [Submission]: {completion}
        ************
        [END DATA]

    Compare the factual content of the submitted answer with the context. \
    Ignore any differences in style, grammar, or punctuation.
    Answer the following questions:
        - Is the Assistant response based only on the context provided? (Y or N)
        - Does the answer include information that is not provided in the context? (Y or N)
        - Is there any disagreement between the response and the context? (Y or N)
        - Count how many questions the user asked. (output a number)
        - For each question that the user asked, is there a corresponding answer to it?
        Question 1: (Y or N)
        Question 2: (Y or N)
        ...
        Question N: (Y or N)
        - Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)
    """

    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]

    response = get_completion_from_messages(messages)
    return response

In [21]:
evaluation_output = eval_with_rubric(necessary_info, original_answer)
print(evaluation_output)

- Is the Assistant response based only on the context provided? (Y or N)
Y

- Does the answer include information that is not provided in the context? (Y or N)
N

- Is there any disagreement between the response and the context? (Y or N)
N

- Count how many questions the user asked. (output a number)
1

- For each question that the user asked, is there a corresponding answer to it?
Question 1: Y

- Of the number of questions asked, how many of these questions were addressed by the answer? (output a number)
1
